In [23]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, FloatSlider
from IPython.display import display

def edh_transform(joint):
    """
    Compute the Expanded Denavit-Hartenberg (EDH) transformation matrix
    using parameters theta, d, alpha, a, r, and b.
    """
    [r, alpha, theta, b, a, d] = joint

    def Rz(theta, d):
        """Rotation and translation matrix about the Z-axis by angle theta."""
        return np.array(
            [
                [np.cos(theta), -np.sin(theta), 0, 0],
                [np.sin(theta), np.cos(theta), 0, 0],
                [0, 0, 1, d],
                [0, 0, 0, 1],
            ]
        )

    def Rx(alpha, a):
        """Rotation and translation matrix about the X-axis by angle alpha."""
        return np.array(
            [
                [1, 0, 0, a],
                [0, np.cos(alpha), -np.sin(alpha), 0],
                [0, np.sin(alpha), np.cos(alpha), 0],
                [0, 0, 0, 1],
            ]
        )

    def Ry(r, b):
        """Additional rotation matrix about the Y-axis by angle r."""
        return np.array(
            [
                [np.cos(r), 0, np.sin(r), 0],
                [0, 1, 0, b],
                [-np.sin(r), 0, np.cos(r), 0],
                [0, 0, 0, 1],
            ]
        )

    # Combine transformations in the order specified by EDH convention
    T = Ry(r, b) @ Rx(alpha, a) @ Rz(theta, d)
    return T

#right motor angles followed by left motor angles
def update_angles(a,b,c,d,e,f,g,h,i,j):
    motor_angle = [a,b,c,d,e,f,g,h,i,j]
    # Define functions for the x, y, and z coordinates based on the values
    right_joint1 = [np.pi, 0, motor_angle[0], 0, -55.3, 0]
    left_joint1 = [0,np.pi,motor_angle[5],0,-55.3,0]
    right_joint2 = [np.pi / 2, -np.pi / 2, motor_angle[1], -35.175, -59.25, 0]
    left_joint2 = [np.pi / 2, np.pi / 2, motor_angle[6], 35.175, -59.25, 0]
    right_joint3 = [np.pi / 2, np.pi / 2, np.pi / 2 + motor_angle[2], 0, -33.125, 0]
    left_joint3 = [np.pi/2,-np.pi/2,-np.pi/2+motor_angle[7],0,-33.125,0]
    right_joint4 = [np.pi, 0, motor_angle[3], 0, 108.5, 0]
    left_joint4 = [np.pi,0,motor_angle[8],0,108.5,0]
    right_joint5 = [0, np.pi, motor_angle[4], 0, 97, 0]
    left_joint5 = [0,np.pi,motor_angle[9],0,97,0]

    l_p0 = [0, 0, 0, 1]
    l_p1 = edh_transform(left_joint1) @ l_p0
    l_p2 = edh_transform(left_joint1) @ edh_transform(left_joint2) @ l_p0
    l_p3 = (
        edh_transform(left_joint1)
        @ edh_transform(left_joint2)
        @ edh_transform(left_joint3)
        @ l_p0
    )
    l_p4 = (
        edh_transform(left_joint1)
        @ edh_transform(left_joint2)
        @ edh_transform(left_joint3)
        @ edh_transform(left_joint4)
        @ l_p0
    )
    l_p5 = (
        edh_transform(left_joint1)
        @ edh_transform(left_joint2)
        @ edh_transform(left_joint3)
        @ edh_transform(left_joint4)
        @ edh_transform(left_joint5)
        @ l_p0
    )

    r_p0 = [0, 0, 0, 1]
    r_p1 = edh_transform(right_joint1) @ r_p0
    r_p2 = edh_transform(right_joint1) @ edh_transform(right_joint2) @ r_p0
    r_p3 = edh_transform(right_joint1) @ edh_transform(right_joint2) @ edh_transform(right_joint3) @ r_p0
    r_p4 = edh_transform(right_joint1) @ edh_transform(right_joint2) @ edh_transform(right_joint3) @ edh_transform(right_joint4) @ r_p0
    r_p5 = edh_transform(right_joint1) @ edh_transform(right_joint2) @ edh_transform(right_joint3) @ edh_transform(right_joint4) @ edh_transform(right_joint5) @ r_p0

    x = []
    y = []
    z = []
    points = [l_p1, l_p2, l_p3, l_p4, l_p5,r_p1, r_p2, r_p3, r_p4, r_p5]
    for p in points:
        x.append(p[0])
        y.append(p[1])
        z.append(p[2])
    return x, y, z


In [ ]:

# Define the range of the sliders
slider_range = (-np.pi / 2, np.pi / 2)

# Initialize five variables with initial values
values = [0] * 10

# Define the function to plot points based on the variables
def plot_3d_points(a,b,c,d,e,f,g,h,i,j):
    x,y,z = update_angles(a,b,c,d,e,f,g,h,i,j)
    
    # Clear the previous plot and create a new one
    plt.figure(figsize=(8, 6))
    ax = plt.axes(projection='3d')
    
    # Plot points and connect them with lines
    ax.plot(x, y, z, marker='o', linestyle='-', color='b', markerfacecolor='r')
    
    # Set axis limits
    ax.set_xlim([-70, 70])
    ax.set_ylim([-40, 40])
    ax.set_zlim([-300, 10])
    
    # Set labels
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    # Show plot
    plt.show()

# Create interactive sliders for each variable
sliders = [
    FloatSlider(value=0, min=slider_range[0], max=slider_range[1], step=0.01, description=f'Var {i+1}')
    for i in range(10)
]

# Create interactive plot
interact(plot_3d_points, a=sliders[0], b=sliders[1], c=sliders[2], d=sliders[3], e=sliders[4],f = sliders[5],g = sliders[6],h = sliders[7],i = sliders[8],j = sliders[9])



interactive(children=(FloatSlider(value=0.0, description='Var 1', max=1.5707963267948966, min=-1.5707963267948…

<function __main__.plot_3d_points(a, b, c, d, e, f, g, h, i, j)>